#### Script to prepare EEG triggers for the memory task of NovMemEEG

In [1]:
# This code creates EEG triggers from E-prime/Excel files 
# import some libraries 
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
SAVE_PATH = '/Enter/your/path/here/'
DATA_PATH = '/Enter/your/path/here/'

In [8]:
def read_txt_file(file_path,run):
    #Create an empty dictionary to store data
    data = {}

    #Open the file in read mode using utf-16 encoding
    with open(file_path, 'r', encoding='utf-16') as f:
        text = f.readlines()
    #Create a generator expression that yields the index of each line that matches '\t\t*** LogFrame Start ***\n'
    if run ==0:     
        start_ind = (i for i, _ in enumerate(text) if _ == '\t\t*** LogFrame Start ***\n')
        end_ind = (i for i, _ in enumerate(text) if _ == '\t\t*** LogFrame End ***\n')
    else:
        start_ind = (i for i, _ in enumerate(text) if _ == '\t*** LogFrame Start ***\n')
        end_ind = (i for i, _ in enumerate(text) if _ == '\t*** LogFrame End ***\n')

    #Loop through each start-end pair and extract data between them
    for i, se in enumerate(zip(start_ind, end_ind)):
        s, e = se
        for row in text[s + 1: e]:
            k, v = row.strip().replace(': ', ':').split(':')
            if k not in data:
                data[k] = [[], []]
            data[k][0].append(v)
            data[k][1].append(i)
    
    new_data = {}
    for k, v in data.items():
        ser = pd.Series(v[0], index=v[1])
        new_data[k] = ser
    return pd.DataFrame(new_data)

In [9]:
def get_triggers(df1, df2):
    # Grab word from study phase 
    Word_idx = df1.columns.get_loc("Word")
    Word = pd.DataFrame(df1.iloc[:,Word_idx])

    #Grab word from memory test 
    Wordoldew_idx = df2.columns.get_loc("WordOldNew")
    Wordoldnew = pd.DataFrame(df2.iloc[:,Wordoldew_idx])

    #Grab whether oldnew in memory test was correct or not 
    OldorNewACC_idx = df2.columns.get_loc("OldOrNew.ACC")
    OldorNewACC = pd.DataFrame(df2.iloc[:,OldorNewACC_idx])

    #Grab the first 60 words in the study phase 
    Word = Word.iloc[:60]
    # add new empty columns to dataframe 
    Word[["EEG_event_label","EEG_event_trigger"]] = ""


    for i in range(len(Word)):
        Wordidx = Word.iloc[i]["Word"]
        index = Wordoldnew.index[Wordoldnew['WordOldNew']==Wordidx].tolist()

        # get the accuracy 
        acc = OldorNewACC.iloc[[index[0]]]
        # get the value of accuracy (so 1.0 or 0.0)
        value = acc['OldOrNew.ACC'].iloc[0]

        #if value == 1.0:
        if value == '1':
            Word["EEG_event_label"].iloc[i]  = "hit"
            Word["EEG_event_trigger"].iloc[i]  = 6
        #elif value == 0.0:
        elif value == '0':
            Word["EEG_event_label"].iloc[i]  = "miss"
            Word["EEG_event_trigger"].iloc[i]  = 5
        else:
            print("no responses given ")
    return Word

In [10]:
def get_file_path(participant_id, day):
    folder_path = os.path.join(DATA_PATH, participant_id, day)
    #Get a list of all the txt files in the folder that start with "NovMem_autorecord_" or "NovMem2022_autorecord_" and end with ".txt"
    txt_files = [f for f in os.listdir(folder_path) if (f.startswith("NovMem_autorecord_") or f.startswith("NovMem2022_autorecord_")) and f.endswith(".txt") and "delayed" not in f]
    #Make sure there is only one txt file in the folder
    assert len(txt_files) == 1, f"Expected 1 txt file but found {len(txt_files)} in {folder_path}"
    
    return os.path.join(folder_path, txt_files[0])

In [11]:
for subIdx in os.listdir(DATA_PATH):
    # Ignore any files in the top-level data directory
    if not os.path.isdir(os.path.join(DATA_PATH, subIdx)):
        continue
    
    day = "DAY2"
    file_path = get_file_path(subIdx, day)

    run = 0
    df1 = read_txt_file(file_path, run)
    run = 1
    df2 = read_txt_file(file_path, run)

    EEG_triggers = get_triggers(df1,df2)

    triggers_dir = os.path.join(SAVE_PATH, subIdx, day)
    os.makedirs(triggers_dir, exist_ok=True)
    excel_file_path = os.path.join(triggers_dir, f"{subIdx}_{day}_MemTask_EEGtriggers.xlsx")

    # Save the dataframe with all EEG triggers to an excel file
    EEG_triggers.to_excel(excel_file_path, index=False)